In [19]:
# Importazioni necessarie per il progetto
import pandas as pd  # Per la gestione dei dati in formato tabellare
import numpy as np  # Per calcoli matematici e distribuzioni statistiche
import dash  # Per creare la dashboard interattiva
from dash import html, dcc  # Per definire il layout e i componenti della dashboard
from dash.dependencies import Input, Output  # Per collegare input e output nei callback
import plotly.graph_objs as go  # Per creare grafici interattivi

# Percorso del file con i dati meteo storici
PERCORSO_FILE_STORICI = 'meteo.xlsx'

# Funzione per calcolare le medie mensili dai dati storici
def calcola_medie_mensili():
    """
    Calcola le medie mensili dei dati climatici storici (temperatura, precipitazioni, umidità, ET0)
    e restituisce una tabella e un dizionario con i valori medi.
    """
    dati = pd.read_excel(PERCORSO_FILE_STORICI)
    dati['mese'] = pd.to_datetime(dati['time']).dt.month
    dati['anno'] = pd.to_datetime(dati['time']).dt.year
    nomi_mesi = ['Gennaio', 'Febbraio', 'Marzo', 'Aprile', 'Maggio', 'Giugno',
                 'Luglio', 'Agosto', 'Settembre', 'Ottobre', 'Novembre', 'Dicembre']
    somme = dati.groupby(['anno', 'mese'])[['precipitation_sum (mm)', 'et0_fao_evapotranspiration (mm)']].sum()
    medie_mensili = pd.DataFrame({
        'Precipitazioni Medie Totali (mm)': [round(somme.groupby('mese')['precipitation_sum (mm)'].mean()[mese], 1) for mese in range(1, 13)],
        'ET0 Medio Totale (mm)': [round(somme.groupby('mese')['et0_fao_evapotranspiration (mm)'].mean()[mese], 1) for mese in range(1, 13)],
        'Temperatura Media (°C)': [round(dati[dati['mese'] == mese]['temperature_2m_mean (°C)'].mean(), 1) for mese in range(1, 13)],
        'ET0 Medio Giornaliero (mm)': [round(dati[dati['mese'] == mese]['et0_fao_evapotranspiration (mm)'].mean(), 1) for mese in range(1, 13)],
        'Umidità Media (%)': [round(dati[dati['mese'] == mese]['relative_humidity_2m_mean (%)'].mean(), 1) for mese in range(1, 13)]
    }, index=nomi_mesi)
    dizionario_medie = {colonna: {mese + 1: medie_mensili[colonna][nomi_mesi[mese]] for mese in range(12)} for colonna in medie_mensili.columns}
    return medie_mensili, dizionario_medie

# Calcolo delle medie mensili e definizione di dizionario_medie
medie_mensili, dizionario_medie = calcola_medie_mensili()

# Funzione per creare una tabella HTML con i dati medi mensili
def crea_tabella_dizionario(dizionario):
    """
    Crea una tabella HTML dal dizionario con i nomi dei mesi come intestazioni.
    """
    nomi_mesi = ['Gennaio', 'Febbraio', 'Marzo', 'Aprile', 'Maggio', 'Giugno',
                 'Luglio', 'Agosto', 'Settembre', 'Ottobre', 'Novembre', 'Dicembre']
    intestazioni = [html.Th(mese, style={'border': '1px solid #ccc', 'padding': '8px'}) for mese in nomi_mesi]
    intestazioni.insert(0, html.Th("Variabile", style={'border': '1px solid #ccc', 'padding': '8px'}))
    righe = []
    for chiave, valore in dizionario.items():
        celle = [html.Td(str(valore[mese]), style={'border': '1px solid #ccc', 'padding': '8px'}) for mese in range(1, 13)]
        celle.insert(0, html.Td(chiave, style={'border': '1px solid #ccc', 'padding': '8px'}))
        righe.append(html.Tr(celle))
    tabella = html.Table([html.Tr(intestazioni)] + righe, style={'borderCollapse': 'collapse', 'width': '100%', 'margin': '20px auto'})
    return tabella
    # Funzione per generare i dati climatici simulati
def genera_dati_climatici_simulati(dati):
    """
    Genera dati climatici simulati (temperatura, precipitazioni, umidità, ET0) per il periodo 2025-2030,
    basandosi sulle medie mensili storiche e sulle distribuzioni statistiche appropriate.
    """
    # Estraggo il mese dai dati storici per calcolare le medie
    dati['mese'] = pd.to_datetime(dati['time']).dt.month
    medie_mensili = dati.groupby('mese').mean()
    deviazioni_mensili = dati.groupby('mese').std()
    
    # Creo un intervallo di date per le previsioni (2025-2030)
    date_previsioni = pd.date_range(start='2025-01-01', end='2030-12-31', freq='D')
    previsioni = pd.DataFrame(index=date_previsioni)

    # Genero i dati climatici simulati per ogni mese
    for mese in range(1, 13):
        giorni_mese = previsioni.index.month == mese

        # Temperatura: Distribuzione Normale
        media_temperatura = medie_mensili.loc[mese, 'temperature_2m_mean (°C)']
        deviazione_temperatura = deviazioni_mensili.loc[mese, 'temperature_2m_mean (°C)']
        previsioni.loc[giorni_mese, 'Temperatura (°C)'] = np.round(
            np.random.normal(media_temperatura, deviazione_temperatura, size=giorni_mese.sum()), 1
        )

        # Precipitazioni: Distribuzione Gamma
        media_precipitazioni = medie_mensili.loc[mese, 'precipitation_sum (mm)']
        theta = media_precipitazioni / 1.0 if media_precipitazioni > 0 else 0.01
        previsioni.loc[giorni_mese, 'Precipitazioni (mm)'] = np.round(
            np.random.gamma(1.0, theta, size=giorni_mese.sum()), 1
        )

        # Umidità: Distribuzione Beta
        media_umidita = medie_mensili.loc[mese, 'relative_humidity_2m_mean (%)'] / 100
        deviazione_umidita = deviazioni_mensili.loc[mese, 'relative_humidity_2m_mean (%)'] / 100
        varianza = min(deviazione_umidita ** 2, media_umidita * (1 - media_umidita) * 0.99)
        if varianza <= 0:
            varianza = 0.01  # Valore minimo per evitare divisioni per zero
        alpha = media_umidita * ((media_umidita * (1 - media_umidita)) / varianza - 1)
        beta = (1 - media_umidita) * ((media_umidita * (1 - media_umidita)) / varianza - 1)
        if alpha <= 0 or beta <= 0:
            alpha = 1
            beta = 1
        previsioni.loc[giorni_mese, 'Umidità (%)'] = np.round(
            np.random.beta(alpha, beta, size=giorni_mese.sum()) * 100, 1
        )

        # ET0: Distribuzione Normale troncata a zero
        media_et0 = medie_mensili.loc[mese, 'et0_fao_evapotranspiration (mm)']
        deviazione_et0 = deviazioni_mensili.loc[mese, 'et0_fao_evapotranspiration (mm)']
        previsioni.loc[giorni_mese, 'ET0 (mm)'] = np.round(
            np.clip(np.random.normal(media_et0, deviazione_et0, size=giorni_mese.sum()), 0, None), 1
        )

    return previsioni

# Genero i dati climatici simulati
dati_iniziali = pd.read_excel(PERCORSO_FILE_STORICI)
previsioni_meteo = genera_dati_climatici_simulati(dati_iniziali)
# Coefficienti colturali (kc) mensili per arance (dal documento Regione Campania, pag. 6)
kc_mensili = {
    1: 0.75,  # Gennaio
    2: 0.75,  # Febbraio
    3: 0.80,  # Marzo
    4: 0.90,  # Aprile
    5: 0.95,  # Maggio
    6: 1.10,  # Giugno
    7: 1.10,  # Luglio
    8: 1.20,  # Agosto
    9: 1.15,  # Settembre
    10: 1.00, # Ottobre
    11: 0.90, # Novembre
    12: 0.75  # Dicembre
}

# Parametri per il calcolo della produzione
superficie = 5  # ettari
numero_alberi = 1250  # Numero di alberi
produzione_per_albero = 150  # kg per albero
frazione_runoff = 0.2  # Frazione di runoff (FAO 56, Capitolo 8, pagina 162)

# Funzione per calcolare la PDF della distribuzione normale usando solo numpy
def norm_pdf(x, mu, sigma):
    """
    Calcola la funzione di densità di probabilità (PDF) della distribuzione normale.
    Args:
        x: Valore del punto in cui calcolare la PDF.
        mu: Media della distribuzione.
        sigma: Deviazione standard della distribuzione.
    Returns:
        Valore della PDF.
    """
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

# Funzione per calcolare il fattore di stress termico K_T
def calcola_kt(temperatura):
    """
    Calcola il fattore di stress termico K_T in base alla temperatura.
    Basato su Regione Campania (2024), pag. 1.
    """
    if temperatura < 10 or temperatura > 35:
        return 0
    elif 10 <= temperatura < 13:
        return (temperatura - 10) / 3
    elif 13 <= temperatura <= 30:
        return 1
    else:  # 30 < temperatura <= 35
        return (35 - temperatura) / 5

# Funzione per calcolare il fattore di stress idrico K_s (senza irrigazione)
def calcola_ks(precipitazioni, et0, kc):
    """
    Calcola il fattore di stress idrico K_s in base al bilancio idrico.
    Basato su Regione Campania (2024), pag. 5.
    """
    if et0 * kc == 0:
        return 1
    else:
        return min(1, precipitazioni / (et0 * kc))

# Funzione per calcolare la produzione giornaliera
def calcola_produzione_giornaliera(previsioni_meteo):
    """
    Calcola la produzione giornaliera di arance con e senza irrigazione,
    basandosi sui dati climatici simulati.
    """
    # Aggiungo colonne per la produzione giornaliera e l'irrigazione
    previsioni_meteo['Irrigazione (m³)'] = 0.0
    previsioni_meteo['Produzione con irrigazione (kg)'] = 0.0
    previsioni_meteo['Produzione senza irrigazione (kg)'] = 0.0

    # Calcolo della produzione giornaliera per ogni stagione
    for anno in range(2025, 2031):
        # Periodo per i dati climatici e irrigazione (tutto l'anno)
        inizio_anno = f'{anno}-04-01'
        fine_anno = f'{anno + 1}-03-31'
        periodo = (previsioni_meteo.index >= inizio_anno) & (previsioni_meteo.index <= fine_anno)

        # Periodo per la produzione (novembre-marzo)
        inizio_stagione = f'{anno}-11-01'
        fine_stagione = f'{anno + 1}-03-31'
        stagione = (previsioni_meteo.index >= inizio_stagione) & (previsioni_meteo.index <= fine_stagione)

        # Calcolo dello stress termico e idrico per l'anno corrente
        kt_sum = 0
        ks_sum = 0
        giorni_totali = 0

        for idx in previsioni_meteo.index[periodo]:
            mese = idx.month
            kc = kc_mensili[mese]
            et0 = previsioni_meteo.loc[idx, 'ET0 (mm)']
            precipitazioni = previsioni_meteo.loc[idx, 'Precipitazioni (mm)']
            temperatura = previsioni_meteo.loc[idx, 'Temperatura (°C)']

            # Calcolo K_T
            kt = calcola_kt(temperatura)
            kt_sum += kt

            # Calcolo K_s
            ks = calcola_ks(precipitazioni, et0, kc)
            ks_sum += ks

            # Calcolo dell'irrigazione
            fabbisogno_idrico = et0 * kc - precipitazioni * (1 - frazione_runoff)
            irrigazione_mm = max(0, fabbisogno_idrico) * 10  # Moltiplico per 10 per ottenere valori realistici (mm -> m³/ha)
            irrigazione_m3 = irrigazione_mm * superficie  # Converto in m³ per l'intera superficie
            previsioni_meteo.loc[idx, 'Irrigazione (m³)'] = irrigazione_m3

            giorni_totali += 1

        # Calcolo delle medie annuali di K_T e K_s per l'anno corrente
        media_kt = kt_sum / giorni_totali if giorni_totali > 0 else 0
        media_ks = ks_sum / giorni_totali if giorni_totali > 0 else 0

        # Calcolo della produzione totale con e senza irrigazione per l'anno corrente
        produzione_totale_con_irr = numero_alberi * produzione_per_albero * media_kt
        produzione_totale_senza_irr = numero_alberi * produzione_per_albero * media_kt * media_ks

        # Calcolo della produzione giornaliera con andamento gaussiano
        if stagione.any():
            # Calcolo del fattore gaussiano per la stagione
            giorni_stagione = (pd.to_datetime(fine_stagione) - pd.to_datetime(inizio_stagione)).days + 1
            mu = 75  # Picco a circa 15 gennaio (75 giorni dall'1 novembre)
            sigma = 30  # Sigma per coprire l'intera stagione
            giorni_dalla_data_inizio = (previsioni_meteo.index - pd.to_datetime(inizio_stagione)).days
            # Calcolo della PDF della distribuzione normale usando solo numpy
            g = norm_pdf(giorni_dalla_data_inizio, mu, sigma)

            # Normalizzo la produzione totale rispetto alla somma di g
            somma_g = np.sum(g[stagione])
            for idx in previsioni_meteo.index[stagione]:
                fattore_gaussiano = g[previsioni_meteo.index.get_loc(idx)]
                # Produzione con irrigazione
                y_d_con_irr = (produzione_totale_con_irr * fattore_gaussiano) / somma_g
                previsioni_meteo.loc[idx, 'Produzione con irrigazione (kg)'] = y_d_con_irr

                # Produzione senza irrigazione
                y_d_senza_irr = (produzione_totale_senza_irr * fattore_gaussiano) / somma_g
                previsioni_meteo.loc[idx, 'Produzione senza irrigazione (kg)'] = y_d_senza_irr

    return previsioni_meteo

# Calcolo la produzione giornaliera
previsioni_meteo = calcola_produzione_giornaliera(previsioni_meteo)
# Creo la dashboard con Dash
dashboard = dash.Dash(__name__)

# Opzioni per il menu a tendina delle stagioni
elenco_stagioni = [
    {'label': '2025/2026', 'value': '2025-04-01_2026-03-31'},
    {'label': '2026/2027', 'value': '2026-04-01_2027-03-31'},
    {'label': '2027/2028', 'value': '2027-04-01_2028-03-31'},
    {'label': '2028/2029', 'value': '2028-04-01_2029-03-31'},
    {'label': '2029/2030', 'value': '2029-04-01_2030-03-31'}
]

# Funzione ausiliaria per calcolare i tick dei grafici
def ottieni_valori_e_etichette(df, nomi_mesi, intervallo_mesi):
    """
    Calcola i valori e le etichette per i tick dei grafici basati su un intervallo di mesi.
    """
    valori_tick = [df.index[df.index.month == m][0] for m in intervallo_mesi]
    return valori_tick, nomi_mesi

# Layout della dashboard
dashboard.layout = html.Div([
    html.H1("Dashboard Produzione Arance", style={'textAlign': 'center', 'color': '#333', 'marginBottom': '20px'}),
    html.Div([
        html.Div([
            html.P("Nome azienda: Terra Verde", style={'fontWeight': 'bold', 'marginBottom': '10px'}),
            html.P("Estensione: 5 ettari", style={'marginBottom': '10px'}),
            html.P("Numero alberi: 1250", style={'marginBottom': '10px'}),
            html.P("Produzione potenziale per albero: circa 150 kg", style={'marginBottom': '10px'}),
            html.P("Produzione potenziale annua: circa 187500 kg", style={'marginBottom': '10px'})
        ], style={'width': '50%', 'padding': '20px', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            html.P(["Scarica i dati storici di Palagonia dal 2010 al 2024. Fonte: ",
                    html.A("Open-Meteo.com", href="https://open-meteo.com/", target="_blank")],
                   style={'marginBottom': '5px'}),
            html.Button("Scarica Dati Storici", id='pulsante_storici', n_clicks=0,
                        style={'marginBottom': '20px', 'padding': '10px', 'fontSize': '16px', 'backgroundColor': '#4CAF50', 
                               'color': 'white', 'border': '2px solid #3d8b40', 'borderRadius': '5px', 'boxShadow': '3px 3px 5px #888888'}),
            dcc.Download(id='scarica_storici'),
            html.P("Scarica i dati medi mensili estrapolati dai dati storici", style={'marginBottom': '5px'}),
            html.Button("Scarica Dati Medi Mensili", id='pulsante_medi', n_clicks=0,
                        style={'marginBottom': '20px', 'padding': '10px', 'fontSize': '16px', 'backgroundColor': 'blue', 
                               'color': 'white', 'border': '2px solid #1a76d2', 'borderRadius': '5px', 'boxShadow': '3px 3px 5px #888888'}),
            dcc.Download(id='scarica_medi'),
            html.P("Scarica le previsioni meteo e produzione dal 2025 al 2030", style={'marginBottom': '5px'}),
            html.Button("Scarica Previsioni 2025-2030", id='pulsante_previsioni', n_clicks=0,
                        style={'marginBottom': '20px', 'padding': '10px', 'fontSize': '16px', 'backgroundColor': '#FF9800', 
                               'color': 'white', 'border': '2px solid #e68900', 'borderRadius': '5px', 'boxShadow': '3px 3px 5px #888888'}),
            dcc.Download(id='scarica_previsioni')
        ], style={'width': '50%', 'padding': '20px', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'flex-start'}),

    html.H3("Dizionario dei dati medi mensili per la simulazione predittiva 2025-2030", 
            style={'textAlign': 'center', 'marginTop': '20px'}),
    crea_tabella_dizionario(dizionario_medie),

    html.H3("Seleziona la stagione di riferimento", style={'textAlign': 'center', 'marginTop': '20px'}),
    dcc.Dropdown(
        id='stagione_selezionata',
        options=elenco_stagioni,
        value='2025-04-01_2026-03-31',
        style={'width': '50%', 'margin': '0 auto 20px auto'}
    ),
    html.Div([
        dcc.Graph(id='grafico_temperatura', style={'width': '100%', 'marginBottom': '20px'}),
        dcc.Graph(id='grafico_precipitazioni', style={'width': '100%', 'marginBottom': '20px'}),
        dcc.Graph(id='grafico_umidita', style={'width': '100%', 'marginBottom': '20px'}),
        dcc.Graph(id='grafico_et0', style={'width': '100%', 'marginBottom': '20px'}),
        dcc.Graph(id='grafico_irrigazione', style={'width': '100%', 'marginBottom': '20px'}),
        dcc.Graph(id='grafico_produzione_con_irrigazione', style={'width': '100%', 'marginBottom': '20px'}),
        dcc.Graph(id='grafico_produzione_senza_irrigazione', style={'width': '100%', 'marginBottom': '20px'})
    ]),
    html.H3("Tabelle dei profitti", style={'textAlign': 'center', 'marginTop': '20px'}),
    html.H4("Produzione con irrigazione", style={'textAlign': 'center'}),
    html.Div(id='tabella_profitti_con_irrigazione', style={'marginBottom': '20px'}),
    html.H4("Produzione potenziale senza irrigazione", style={'textAlign': 'center'}),
    html.Div(id='tabella_profitti_senza_irrigazione'),
    # Spazio bianco di altezza 300 px
    html.Div(style={'height': '300px'})
], style={'padding': '20px', 'fontFamily': 'Arial', 'maxWidth': '1200px', 'margin': 'auto'})

# Callback per scaricare i file
@dashboard.callback(
    Output('scarica_storici', 'data'),
    Input('pulsante_storici', 'n_clicks'),
    prevent_initial_call=True
)
def scarica_file_storici(n_clicks):
    return dcc.send_file(PERCORSO_FILE_STORICI)

@dashboard.callback(
    Output('scarica_medi', 'data'),
    Input('pulsante_medi', 'n_clicks'),
    prevent_initial_call=True
)
def scarica_file_medi(n_clicks):
    return dcc.send_file('medie.xlsx')

@dashboard.callback(
    Output('scarica_previsioni', 'data'),
    Input('pulsante_previsioni', 'n_clicks'),
    prevent_initial_call=True
)
def scarica_file_previsioni(n_clicks):
    return dcc.send_file('previsioni_2025_2030.xlsx')

# Callback per aggiornare i grafici e le tabelle
@dashboard.callback(
    [Output('grafico_temperatura', 'figure'),
     Output('grafico_precipitazioni', 'figure'),
     Output('grafico_umidita', 'figure'),
     Output('grafico_et0', 'figure'),
     Output('grafico_irrigazione', 'figure'),
     Output('grafico_produzione_con_irrigazione', 'figure'),
     Output('grafico_produzione_senza_irrigazione', 'figure'),
     Output('tabella_profitti_con_irrigazione', 'children'),
     Output('tabella_profitti_senza_irrigazione', 'children')],
    Input('stagione_selezionata', 'value')
)
def aggiorna_grafico_e_tabelle(stagione_selezionata):
    data_inizio, data_fine = stagione_selezionata.split('_')
    anno_inizio = int(data_inizio[:4])
    anno_fine = int(data_fine[:4])
    
    # Filtro i dati giornalieri per i grafici climatici e irrigazione (tutto l'anno)
    df_stagione = previsioni_meteo[data_inizio:data_fine]
    date_italiano = df_stagione.index.strftime('%d/%m/%Y')

    # Filtro i dati giornalieri per i grafici della produzione (solo novembre-marzo)
    inizio_produzione = f'{anno_inizio}-11-01'
    fine_produzione = f'{anno_inizio + 1}-03-31'
    df_produzione = previsioni_meteo[inizio_produzione:fine_produzione]

    # Tick per i grafici climatici e irrigazione (tutto l'anno)
    tickvals_clima, ticktext_clima = ottieni_valori_e_etichette(
        df_stagione, 
        ['Aprile', 'Maggio', 'Giugno', 'Luglio', 'Agosto', 'Settembre', 'Ottobre', 'Novembre', 'Dicembre', 'Gennaio', 'Febbraio', 'Marzo'],
        [4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]
    )

    # Tick per i grafici della produzione (novembre-marzo)
    tickvals_produzione, ticktext_produzione = ottieni_valori_e_etichette(
        df_produzione, 
        ['Novembre', 'Dicembre', 'Gennaio', 'Febbraio', 'Marzo'],
        [11, 12, 1, 2, 3]
    )

    # Grafico Temperatura
    grafico_temperatura = go.Figure()
    grafico_temperatura.add_trace(go.Scatter(
        x=df_stagione.index,
        y=df_stagione['Temperatura (°C)'],
        mode='lines',
        name='Temperatura (°C)',
        line={'color': '#FF5733'},
        customdata=date_italiano,
        hovertemplate='%{customdata}: %{y} °C'
    ))
    grafico_temperatura.update_layout(
        title=f'Temperatura (°C) Stagione {anno_inizio}/{anno_fine}',
        xaxis_title='Mese',
        yaxis_title='Gradi Celsius (°C)',
        xaxis={'tickvals': tickvals_clima, 'ticktext': ticktext_clima},
        height=300,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Grafico Precipitazioni
    grafico_precipitazioni = go.Figure()
    grafico_precipitazioni.add_trace(go.Scatter(
        x=df_stagione.index,
        y=df_stagione['Precipitazioni (mm)'],
        mode='lines',
        name='Precipitazioni (mm)',
        line={'color': '#33CCFF'},
        customdata=date_italiano,
        hovertemplate='%{customdata}: %{y} mm'
    ))
    grafico_precipitazioni.update_layout(
        title=f'Precipitazioni (mm) Stagione {anno_inizio}/{anno_fine}',
        xaxis_title='Mese',
        yaxis_title='Millimetri (mm)',
        xaxis={'tickvals': tickvals_clima, 'ticktext': ticktext_clima},
        height=300,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Grafico Umidità
    grafico_umidita = go.Figure()
    grafico_umidita.add_trace(go.Scatter(
        x=df_stagione.index,
        y=df_stagione['Umidità (%)'],
        mode='lines',
        name='Umidità (%)',
        line={'color': '#33FF57'},
        customdata=date_italiano,
        hovertemplate='%{customdata}: %{y} %'
    ))
    grafico_umidita.update_layout(
        title=f'Umidità (%) Stagione {anno_inizio}/{anno_fine}',
        xaxis_title='Mese',
        yaxis_title='Percentuale (%)',
        xaxis={'tickvals': tickvals_clima, 'ticktext': ticktext_clima},
        height=300,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Grafico ET0
    grafico_et0 = go.Figure()
    grafico_et0.add_trace(go.Scatter(
        x=df_stagione.index,
        y=df_stagione['ET0 (mm)'],
        mode='lines',
        name='ET0 (mm)',
        line={'color': '#FF33CC'},
        customdata=date_italiano,
        hovertemplate='%{customdata}: %{y} mm'
    ))
    grafico_et0.update_layout(
        title=f'Evapotraspirazione (ET0) (mm) Stagione {anno_inizio}/{anno_fine}',
        xaxis_title='Mese',
        yaxis_title='Millimetri (mm)',
        xaxis={'tickvals': tickvals_clima, 'ticktext': ticktext_clima},
        height=300,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Grafico Irrigazione
    grafico_irrigazione = go.Figure()
    grafico_irrigazione.add_trace(go.Scatter(
        x=df_stagione.index,
        y=df_stagione['Irrigazione (m³)'],
        mode='lines',
        name='Irrigazione (m³)',
        line={'color': '#00CED1'},
        customdata=date_italiano,
        hovertemplate='%{customdata}: %{y} m³'
    ))
    grafico_irrigazione.update_layout(
        title=f'Irrigazione (m³) Stagione {anno_inizio}/{anno_fine}',
        xaxis_title='Mese',
        yaxis_title='Metri Cubi (m³)',
        xaxis={'tickvals': tickvals_clima, 'ticktext': ticktext_clima},
        height=300,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Grafico Produzione con irrigazione 
    date_italiano_produzione = df_produzione.index.strftime('%d/%m/%Y')
    grafico_produzione_con_irr = go.Figure()
    grafico_produzione_con_irr.add_trace(go.Scatter(
        x=df_produzione.index,
        y=df_produzione['Produzione con irrigazione (kg)'],
        mode='lines',
        name='Produzione con irrigazione (kg)',
        line={'color': '#FFA500'},
        customdata=date_italiano_produzione,
        hovertemplate='%{customdata}: %{y:.1f} kg'
    ))
    grafico_produzione_con_irr.update_layout(
        title=f'Produzione Giornaliera di Arance con Irrigazione ({anno_inizio}/{anno_fine})',
        xaxis_title='Data',
        yaxis_title='Produzione (kg)',
        xaxis={'tickvals': tickvals_produzione, 'ticktext': ticktext_produzione},
        height=500,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Grafico Produzione senza irrigazione 
    grafico_produzione_senza_irr = go.Figure()
    grafico_produzione_senza_irr.add_trace(go.Scatter(
        x=df_produzione.index,
        y=df_produzione['Produzione senza irrigazione (kg)'],
        mode='lines',
        name='Produzione senza irrigazione (kg)',
        line={'color': '#FF4500'},
        customdata=date_italiano_produzione,
        hovertemplate='%{customdata}: %{y:.1f} kg'
    ))
    grafico_produzione_senza_irr.update_layout(
        title=f'Produzione Giornaliera di Arance senza Irrigazione ({anno_inizio}/{anno_fine})',
        xaxis_title='Data',
        yaxis_title='Produzione (kg)',
        xaxis={'tickvals': tickvals_produzione, 'ticktext': ticktext_produzione},
        height=500,
        margin={'l': 100, 'r': 20, 't': 50, 'b': 50}
    )

    # Calcolo dati per le tabelle (produzione: novembre-marzo; irrigazione: tutto l'anno)
    produzione_totale_con_irrigazione = df_produzione['Produzione con irrigazione (kg)'].sum()
    produzione_totale_senza_irrigazione = df_produzione['Produzione senza irrigazione (kg)'].sum()
    prezzo_vendita = 0.50  # €/kg (aggiornato a 50 centesimi)
    ricavo_con_irrigazione = produzione_totale_con_irrigazione * prezzo_vendita
    ricavo_senza_irrigazione = produzione_totale_senza_irrigazione * prezzo_vendita
    irrigazione_totale = df_stagione['Irrigazione (m³)'].sum()  # Considero tutto l'anno
    costo_m3 = 0.40  # €/m³
    costi_irrigazione = irrigazione_totale * costo_m3
    profitto_con_irrigazione = ricavo_con_irrigazione - costi_irrigazione
    profitto_senza_irrigazione = ricavo_senza_irrigazione

    # Tabella con irrigazione
    tabella_con_irrigazione = html.Table([
        html.Tr([html.Th(col, style={'border': '1px solid #ccc', 'padding': '8px'}) for col in [
            "Produzione Totale (kg)", "Prezzo Vendita (€/kg)", "Ricavo Potenziale (€)", 
            "Irrigazione Totale (m³)", "Costo al m³ (€)", "Costi Irrigazione (€)", "Profitto (€)"
        ]]),
        html.Tr([html.Td(str(round(produzione_totale_con_irrigazione, 1)), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(prezzo_vendita), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(round(ricavo_con_irrigazione, 2)), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(round(irrigazione_totale, 1)), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(costo_m3), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(round(costi_irrigazione, 2)), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(round(profitto_con_irrigazione, 2)), style={'border': '1px solid #ccc', 'padding': '8px'})])
    ], style={'borderCollapse': 'collapse', 'width': '100%', 'margin': '20px auto'})

    # Tabella senza irrigazione
    tabella_senza_irrigazione = html.Table([
        html.Tr([html.Th(col, style={'border': '1px solid #ccc', 'padding': '8px'}) for col in [
            "Produzione Totale (kg)", "Prezzo Vendita (€/kg)", "Ricavo Potenziale (€)", 
            "Irrigazione Totale (m³)", "Costo al m³ (€)", "Costi Irrigazione (€)", "Profitto (€)"
        ]]),
        html.Tr([html.Td(str(round(produzione_totale_senza_irrigazione, 1)), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(prezzo_vendita), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(round(ricavo_senza_irrigazione, 2)), style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td("0", style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td("0", style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td("0", style={'border': '1px solid #ccc', 'padding': '8px'}),
                 html.Td(str(round(profitto_senza_irrigazione, 2)), style={'border': '1px solid #ccc', 'padding': '8px'})])
    ], style={'borderCollapse': 'collapse', 'width': '100%', 'margin': '20px auto'})

    return (grafico_temperatura, grafico_precipitazioni, grafico_umidita, grafico_et0, 
            grafico_irrigazione, grafico_produzione_con_irr, grafico_produzione_senza_irr, 
            tabella_con_irrigazione, tabella_senza_irrigazione)

# Avvio la dashboard
if __name__ == '__main__':
    dashboard.run(debug=True, port=8994)
